# Imports & Dependencies
- Package Requirements: pandas, numpy, matplotlib, shap, scikit-learn
- Additional files: collinearity_check.py & kennard_stone.py

In [1]:
# Basic Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# External Python Files
import kennard_stone as ks  # Comment out for random split only
from collinearity_check import assess_collinearity

# Plot Settings
font = {'family': 'Avenir Next', 'weight': 'normal', 'size': 16}
plt.rc('font', **font)

random_state = 42  # For reproducibility


# Data Preparation
- Fill out the following cell to load data. Requests a single header row.
- Optional Visualization of Data Distribution (Histogram)
- Optional Binarizing of Continuous Data (Manual Threshold)
- Collinearity Filtering of Features


In [2]:
excel_file = "Example.xlsx" # Excel name as string
sheet_name = "Model" # sheet with training data and features
index_name = 'Index' # numerical column

n_features = 190 # number of independent variables
feature_start_col = 2 # zero-indexed
output_column = 'Average_R2R3' #string value for output column
output_binary = False # boolean for whether the output is binary. False = continuous

# Load Excel spreadsheet as DataFrame
data = pd.read_excel(excel_file, sheet_name=sheet_name, index_col=index_name)

# Define features and output variables
X = data.iloc[:, feature_start_col:n_features+feature_start_col]
if output_binary == True:
    y_binary = data[output_column]
else:
    y_cont = data[output_column]

display(data.head())

,kraken_id,Average_R2R3,vmin_vmin_boltz,vmin_r_boltz,fmo_e_homo_boltz,fmo_e_lumo_boltz,fmo_mu_boltz,fmo_eta_boltz,fmo_omega_boltz,somo_ra_boltz,...,sterimol_burB5_boltz,sterimol_burB5_min,sterimol_burB5_max,sterimol_burB5_delta,sterimol_burB5_vburminconf,sterimol_burL_boltz,sterimol_burL_min,sterimol_burL_max,sterimol_burL_delta,sterimol_burL_vburminconf
Index,,,,,,,,,,,,,,,,,,,,,
0,2014,0.0,-0.075848,1.822230,-0.208145,-0.028543,-0.118344,0.179602,0.038990,0.050703,...,7.051159,6.796484,7.684267,0.887783,6.966066,7.107802,7.032582,7.719974,0.687392,7.480646
1,2032,0.0,-0.070351,1.830065,-0.211892,-0.049081,-0.130486,0.162811,0.052314,0.033953,...,7.188534,6.969260,7.705582,0.736322,7.038297,7.115989,7.002841,7.339487,0.336646,7.101796
2,39,5.8,-0.065696,1.830797,-0.214109,-0.037421,-0.125765,0.176688,0.044763,0.046666,...,6.959986,6.910482,7.031886,0.121404,7.031886,6.898491,6.727968,7.351934,0.623966,7.269082
3,31,0.0,-0.062160,1.782107,-0.225350,-0.025040,-0.125195,0.200310,0.039124,0.080490,...,6.029335,6.029335,6.029335,0.000000,6.029335,7.042766,7.042766,7.042766,0.000000,7.042766
4,12,0.2,-0.062406,1.780166,-0.221080,0.028890,-0.096095,0.249970,0.018471,0.106560,...,6.198222,6.198222,6.198222,0.000000,6.198222,6.665977,6.665977,6.665977,0.000000,6.665977


## Convert Continuous Data to Binary 
- Running this cell will not impact binary input data
- Only prompts input if inputting continuous data

In [3]:
# interactive threshold for binarizing continuous data
# y_binary only exists if boolean set to True
try:
    _ = y_binary
except NameError:
    user_threshold = float(input("Enter Threshold (% Yield, etc.):"))
    print('Binarizing Data Based on User Threshold Value...')
    y_binary = (y_cont > user_threshold).astype(int)
    y_binary_df = pd.Series.to_frame(y_binary)
    #display(y_binary_df)
else:
    print('Binary data already imported! Proceed with feature selection.')


Binarizing Data Based on User Threshold Value...


## Interactive Collinearity Cutoff
- simple pairwise collinearity threshold (Note: External file needed)
- Note: downstream SVM incorporates recursive feature elimination

In [4]:
#redefine x after removing independent variables with all constant values
X_no_const = X.loc[:, X.apply(pd.Series.nunique) > 1]

#enter threshold (interactive)
threshold = float(input("Enter the collinearity threshold (e.g., 0.7): "))
collinear_features = assess_collinearity(X_no_const, threshold)

#redfine x after dropping collinear features
X_nocolin = X_no_const.drop(columns=collinear_features)

#print shape of remaining dataframe
print("Shape of X_nocolin:", X_nocolin.shape)
display(X_nocolin.head())

Shape of X_nocolin: (99, 141)


,vmin_vmin_boltz,vmin_r_boltz,fmo_e_homo_boltz,fmo_e_lumo_boltz,fmo_mu_boltz,fmo_eta_boltz,somo_ra_boltz,somo_rc_boltz,nbo_P_boltz,nbo_P_ra_boltz,...,sterimol_burB5_boltz,sterimol_burB5_min,sterimol_burB5_max,sterimol_burB5_delta,sterimol_burB5_vburminconf,sterimol_burL_boltz,sterimol_burL_min,sterimol_burL_max,sterimol_burL_delta,sterimol_burL_vburminconf
Index,,,,,,,,,,,,,,,,,,,,,
0,-0.075848,1.822230,-0.208145,-0.028543,-0.118344,0.179602,0.050703,-0.348867,0.818007,0.772409,...,7.051159,6.796484,7.684267,0.887783,6.966066,7.107802,7.032582,7.719974,0.687392,7.480646
1,-0.070351,1.830065,-0.211892,-0.049081,-0.130486,0.162811,0.033953,-0.352418,0.808347,0.782050,...,7.188534,6.969260,7.705582,0.736322,7.038297,7.115989,7.002841,7.339487,0.336646,7.101796
2,-0.065696,1.830797,-0.214109,-0.037421,-0.125765,0.176688,0.046666,-0.361883,0.781730,0.769078,...,6.959986,6.910482,7.031886,0.121404,7.031886,6.898491,6.727968,7.351934,0.623966,7.269082
3,-0.062160,1.782107,-0.225350,-0.025040,-0.125195,0.200310,0.080490,-0.404680,0.765850,0.721410,...,6.029335,6.029335,6.029335,0.000000,6.029335,7.042766,7.042766,7.042766,0.000000,7.042766
4,-0.062406,1.780166,-0.221080,0.028890,-0.096095,0.249970,0.106560,-0.449980,0.751860,0.729910,...,6.198222,6.198222,6.198222,0.000000,6.198222,6.665977,6.665977,6.665977,0.000000,6.665977


## Training Test Split

In [5]:
# Perform train/test split (random)
test_ratio = 0.20 # in percent (e.g., 0.20 = 20% test, 80% train)
X_train, X_test, y_train, y_test = ks.train_test_split(X_nocolin, y_binary, test_size=test_ratio,random_state=random_state)

# Apply scaling to training set and transform test based on train mean and stddev
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled arrays back to dataframes
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Decision Tree Classifier

In [6]:
dt_classifier = DecisionTreeClassifier(random_state=random_state)

param_grid = {
    'max_depth': [1, 2, 3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 5],
    'criterion': ['gini', 'entropy', 'log_loss']
}

grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=False)
grid_search.fit(X_train_scaled, y_train)

best_dt_classifier = grid_search.best_estimator_

y_train_pred = best_dt_classifier.predict(X_train_scaled)
y_test_pred = best_dt_classifier.predict(X_test_scaled)

# Evaluate the model
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
cross_val_accuracy = cross_val_score(best_dt_classifier, X_train_scaled, y_train, cv=5).mean()
train_f1 = f1_score(y_train, y_train_pred, average='weighted')
test_f1 = f1_score(y_test, y_test_pred, average='weighted')

# Print results
print("Best parameters found: ", grid_search.best_params_)
print("Train Accuracy: ", train_accuracy)
print("Test Accuracy: ", test_accuracy)
print("5-Fold Cross-Validation Accuracy: ", cross_val_accuracy)
print("Train F1-Score: ", train_f1)
print("Test F1-Score: ", test_f1)
print("Classification report for test set:\n", classification_report(y_test, y_test_pred))

Best parameters found:  {'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
Train Accuracy:  0.9493670886075949
Test Accuracy:  0.8
5-Fold Cross-Validation Accuracy:  0.8858333333333335
Train F1-Score:  0.9467827004219409
Test F1-Score:  0.7866666666666666
Classification report for test set:
               precision    recall  f1-score   support

           0       0.81      0.93      0.87        14
           1       0.75      0.50      0.60         6

    accuracy                           0.80        20
   macro avg       0.78      0.71      0.73        20
weighted avg       0.79      0.80      0.79        20



# Random Forest Classifier

In [7]:
rf_classifier = RandomForestClassifier(random_state=random_state)

param_grid = {
    'n_estimators': [10, 20, 25, 50, 100, 200],
    'max_depth': [1, 3, 5, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4, 5],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=False)
grid_search.fit(X_train_scaled, y_train)

best_rf_classifier = grid_search.best_estimator_

y_train_pred = best_rf_classifier.predict(X_train_scaled)
y_test_pred = best_rf_classifier.predict(X_test_scaled)

# Evaluate the model
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
cross_val_accuracy = cross_val_score(best_rf_classifier, X_train_scaled, y_train, cv=5).mean()
train_f1 = f1_score(y_train, y_train_pred, average='weighted')
test_f1 = f1_score(y_test, y_test_pred, average='weighted')

# Print results
print("Best parameters found: ", grid_search.best_params_)
print("Train Accuracy: ", train_accuracy)
print("Test Accuracy: ", test_accuracy)
print("5-Fold Cross-Validation Accuracy: ", cross_val_accuracy)
print("Train F1-Score: ", train_f1)
print("Test F1-Score: ", test_f1)
print("Classification report for test set:\n", classification_report(y_test, y_test_pred))

Best parameters found:  {'bootstrap': True, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 25}
Train Accuracy:  0.9746835443037974
Test Accuracy:  0.8
5-Fold Cross-Validation Accuracy:  0.9108333333333334
Train F1-Score:  0.9740933293148483
Test F1-Score:  0.7625
Classification report for test set:
               precision    recall  f1-score   support

           0       0.78      1.00      0.88        14
           1       1.00      0.33      0.50         6

    accuracy                           0.80        20
   macro avg       0.89      0.67      0.69        20
weighted avg       0.84      0.80      0.76        20



/opt/miniconda3/envs/modeling/lib/python3.11/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


# Logistic Regression

In [8]:
lr_classifier = LogisticRegression(random_state=random_state, max_iter=10000)

param_grid = {
    'penalty': ['elasticnet'],
    'C': [0.01, 0.1, 1, 5, 10, 100],
    'solver': ['saga'],
    'l1_ratio': [0.5]  # Only used if penalty is 'elasticnet'
}

grid_search = GridSearchCV(estimator=lr_classifier, param_grid=param_grid, cv=5, n_jobs=-1, verbose=False)
grid_search.fit(X_train_scaled, y_train)

best_lr_classifier = grid_search.best_estimator_

y_train_pred = best_lr_classifier.predict(X_train_scaled)
y_test_pred = best_lr_classifier.predict(X_test_scaled)

# Evaluate the model
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
cross_val_accuracy = cross_val_score(best_lr_classifier, X_train_scaled, y_train, cv=5).mean()
train_f1 = f1_score(y_train, y_train_pred, average='weighted')
test_f1 = f1_score(y_test, y_test_pred, average='weighted')

# Print results
print("Best parameters found: ", grid_search.best_params_)
print("Train Accuracy: ", train_accuracy)
print("Test Accuracy: ", test_accuracy)
print("5-Fold Cross-Validation Accuracy: ", cross_val_accuracy)
print("Train F1-Score: ", train_f1)
print("Test F1-Score: ", test_f1)
print("Classification report for test set:\n", classification_report(y_test, y_test_pred))

Best parameters found:  {'C': 5, 'l1_ratio': 0.5, 'penalty': 'elasticnet', 'solver': 'saga'}
Train Accuracy:  1.0
Test Accuracy:  0.85
5-Fold Cross-Validation Accuracy:  0.8608333333333335
Train F1-Score:  1.0
Test F1-Score:  0.8457680250783699
Classification report for test set:
               precision    recall  f1-score   support

           0       0.87      0.93      0.90        14
           1       0.80      0.67      0.73         6

    accuracy                           0.85        20
   macro avg       0.83      0.80      0.81        20
weighted avg       0.85      0.85      0.85        20



# 

# Support Vector Machines

In [9]:
# SVM Grid Search Parameters
k_fold = 5  
c_values = [0.1, 0.5, 1, 5, 10]  
gamma_values = ['scale', 'auto']  
degree_values = [2, 3, 4]  

param_grids = {
    'linear': {'svc__C': c_values},
    'poly': {'svc__C': c_values, 'svc__gamma': gamma_values, 'svc__degree': degree_values},
    'rbf': {'svc__C': c_values, 'svc__gamma': gamma_values},
    'sigmoid': {'svc__C': c_values, 'svc__gamma': gamma_values}
}

selected_kernels = ['linear', 'poly', 'rbf', 'sigmoid']
best_models = {}

# Iterate over each selected kernel and perform GridSearchCV
for kernel in selected_kernels:
    print(f"Running GridSearchCV for {kernel} kernel...")

    svm = SVC(kernel=kernel, random_state=random_state)
    pipeline = Pipeline([('svc', svm)])

    grid_search = GridSearchCV(pipeline, param_grids[kernel], cv=k_fold, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train_scaled_df, y_train)

    best_pipeline = grid_search.best_estimator_
    best_models[kernel] = {
        'best_pipeline': best_pipeline,
        'best_svc_model': best_pipeline.named_steps['svc'],
    }

    y_train_pred = best_pipeline.predict(X_train_scaled_df)
    y_test_pred = best_pipeline.predict(X_test_scaled_df)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')

    # Print results for the current kernel
    print(f"Best {kernel.capitalize()} SVM Parameters:", best_pipeline.named_steps['svc'].get_params())
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("Training F1-Score:", train_f1)
    print("Test F1-Score:", test_f1)
    print("5-Fold Cross-Validation Accuracy:", cross_val_score(best_pipeline, X_train_scaled_df, y_train, cv=k_fold).mean())
    print("Classification Report (Training):")
    print(classification_report(y_train, y_train_pred))
    print("Classification Report (Test):")
    print(classification_report(y_test, y_test_pred))
    print("")


Running GridSearchCV for linear kernel...
Best Linear SVM Parameters: {'C': 0.1, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'probability': False, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Training Accuracy: 0.9873417721518988
Test Accuracy: 0.75
Training F1-Score: 0.9872006137322593
Test F1-Score: 0.7204301075268817
5-Fold Cross-Validation Accuracy: 0.8358333333333334
Classification Report (Training):
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        62
           1       1.00      0.94      0.97        17

    accuracy                           0.99        79
   macro avg       0.99      0.97      0.98        79
weighted avg       0.99      0.99      0.99        79

Classification Report (Test):
              precision    recall  f1-score   support

           0       0